In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/banknote-authenticationcsv/BankNote_Authentication.csv


In [4]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import accuracy_score

# 1. Load dataset
file_path = "/kaggle/input/banknote-authenticationcsv/BankNote_Authentication.csv"
data = pd.read_csv(file_path)

# Features and target
X = data.drop('class', axis=1).values
y = data['class'].values

# Handle class imbalance by upsampling minority class (manual SMOTE alternative)
df = pd.DataFrame(data)
majority = df[df['class']==0]
minority = df[df['class']==1]
minority_upsampled = resample(minority,
                              replace=True,
                              n_samples=len(majority),
                              random_state=42)
df_balanced = pd.concat([majority, minority_upsampled])

X_res = df_balanced.drop('class', axis=1).values
y_res = df_balanced['class'].values

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to tensors
tf_X_train = tf.constant(X_train, dtype=tf.float32)
tf_y_train = tf.constant(y_train.reshape(-1,1), dtype=tf.float32)
tf_X_test = tf.constant(X_test, dtype=tf.float32)
tf_y_test = tf.constant(y_test.reshape(-1,1), dtype=tf.float32)

# 2. Build a simple ANN
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# 3. Compile model with SGD optimizer and binary cross-entropy
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 4. Train model
history = model.fit(tf_X_train, tf_y_train, epochs=10, batch_size=32, verbose=1)

# 5. Predictions on test set
y_pred_prob = model.predict(tf_X_test)
y_pred_class = (y_pred_prob > 0.5).astype(int)

# 6. Evaluate accuracy
acc = accuracy_score(y_test, y_pred_class)
print(f"\nTest Accuracy after balancing and ANN training: {acc:.4f}")

# 7. Show some sample predictions
print("\nSample Predictions vs Actual:")
for i in range(5):
    print(f"  Predicted: {y_pred_class[i][0]}, Actual: {y_test[i]}")

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6409 - loss: 0.6028 
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9347 - loss: 0.3650
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9800 - loss: 0.2177
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9803 - loss: 0.1376
Epoch 5/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9915 - loss: 0.0917
Epoch 6/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9869 - loss: 0.0655
Epoch 7/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9826 - loss: 0.0475
Epoch 8/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9915 - loss: 0.0314
Epoch 9/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9881 - loss: 0.0294
Epoch 10/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9887 - loss: 0.0263
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

Test Accuracy after balancing and ANN training: 0.9902

Sample Predictions vs Actual:
  Predicted: 1, Actual: 1
  Predicted: 0, A